#Comparacion de Fondos de Pensiones a Diciembre 2025

In [ ]:
import requests, re, os
from urllib.parse import urljoin

url = "https://www.spensiones.cl/apps/centroEstadisticas/paginaCuadrosCCEE.php?menu=sest&menuN1=sistpens&menuN2=fondospen"
carpeta = "/content/drive/MyDrive/FondosdePensiones/excels_fondos_pensiones"
os.makedirs(carpeta, exist_ok=True)

headers = {
    "User-Agent": "Mozilla/5.0 (compatible; DataBot/1.0)"
}

html = requests.get(url, headers=headers, timeout=20).text
rutas = re.findall(r"document\.location\.href='([^']+\.xls)'", html)
links = [urljoin("https://www.spensiones.cl", r) for r in rutas]

print(f"Encontrados {len(links)} archivos")

for link in links:
    nombre = link.split("/")[-1]
    try:
        print("Descargando:", nombre)
        r = requests.get(link, headers=headers, timeout=30)
        r.raise_for_status()
        with open(f"{carpeta}/{nombre}", "wb") as f:
            f.write(r.content)
    except Exception as e:
        print("Error con", nombre, ":", e)

print("Proceso terminado.")




Encontrados 12 archivos
Descargando: activos_fondos_pensiones.xls
Descargando: pasivos_fondos_pensiones.xls
Descargando: variacion_patrimonial_fondos_pensiones.xls
Descargando: valor_fondos_pensiones.xls
Descargando: activos_fondos_pensiones_anuales.xls
Descargando: evolucion_inversion_fondos_pensiones_sector_institucional_instrumentos_financieros.xls
Descargando: rentabilidad_real_anual_fondo_pensiones_tipo_a_deflactada_uf.xls
Descargando: rentabilidad_real_anual_fondo_pensiones_tipo_b_deflactada_uf.xls
Descargando: rentabilidad_real_anual_fondo_pensiones_tipo_c_deflactada_uf.xls
Descargando: rentabilidad_real_anual_fondo_pensiones_tipo_d_deflactada_uf.xls
Descargando: rentabilidad_real_anual_fondo_pensiones_tipo_e_deflactada_uf.xls
Descargando: rentabilidad_real_mensual_fondos_deflactada_uf.xls
Proceso terminado.


In [ ]:
import requests
import pandas as pd
from io import BytesIO
import plotly.graph_objects as go

# -------------------------------------------------
# 1. Leer archivo Excel
# -------------------------------------------------
file_path = "/content/drive/MyDrive/FondosdePensiones/rentabilidad_real_mensual_fondos_deflactada_uf.xls"
df = pd.read_excel(file_path, skiprows=3, engine="xlrd")

# ----------------------------------------
# 2. Renombrar columnas
# ----------------------------------------
df.columns = ['Fecha_raw','A','B','C','D','E']

# ----------------------------------------
# 3. Convertir fechas y eliminar inválidas
# ----------------------------------------
df['Fecha_dt'] = pd.to_datetime(df['Fecha_raw'], errors='coerce')
df = df.dropna(subset=['Fecha_dt'])

# ----------------------------------------
# 4. Filtrar período 2021–2025
# ----------------------------------------
df = df[(df['Fecha_dt'] >= '2021-01-01') & (df['Fecha_dt'] <= '2025-12-31')]

# ----------------------------------------
# 5. Definir fondos y nombres
# ----------------------------------------
fondos = ['A','B','C','D','E']
nombres = {
    'A':'Fondo A Más riesgoso',
    'B':'Fondo B Riesgoso',
    'C':'Fondo C Intermedio',
    'D':'Fondo D Conservador',
    'E':'Fondo E Más conservador'
}

# ----------------------------------------
# 6. Pasar % a decimal
# ----------------------------------------
df[fondos] = df[fondos] / 100

# ----------------------------------------
# 7. Calcular rentabilidad acumulada
# ----------------------------------------
df_acum = df.copy()
for f in fondos:
    df_acum[f] = (1 + df_acum[f]).cumprod() - 1   # acumulado en decimal
    df_acum[f] = (df_acum[f] * 100).round(2)      # pasar a %

# ----------------------------------------
# 8. Definir ticks en junio mostrando solo año
# ----------------------------------------
ticks_x = df_acum[df_acum['Fecha_dt'].dt.month == 6]['Fecha_dt']
ticks_text = ticks_x.dt.year.astype(str)

# ----------------------------------------
# 9. Crear figura
# ----------------------------------------
fig = go.Figure()

# Agregar una línea por cada fondo
for f in fondos:
    fig.add_trace(go.Scatter(
        x=df_acum['Fecha_dt'],
        y=df_acum[f],
        mode='lines',
        name=nombres[f],
        hovertemplate=f"{nombres[f]}<br>Fecha: %{{x|%b-%Y}}<br>Rentabilidad: %{{y:.2f}}%<extra></extra>"
    ))

# Línea horizontal en 0%
fig.add_hline(y=0)

# ----------------------------------------
# 10. Formato del gráfico
# ----------------------------------------
fig.update_layout(
    height=800,
    width=1200,
    title=dict(
        text="Rentabilidad Real Acumulada de los Fondos de Pensiones<br>(Periodo enero 2021 - diciembre 2025)",
        x=0.5,
        xanchor="center",
        font=dict(size=22)
    ),
    yaxis=dict(
        range=[-25,25],
        autorange=False,
        tickmode="array",
        tickvals=[-25,-20,-15,-10,-5,0,5,10,15,20,25],
        ticksuffix="%",
        showgrid=True,
        zeroline=True,
        ticklabelstandoff=10,
        tickfont=dict(size=14)
    ),
    xaxis=dict(
        tickmode="array",
        tickvals=ticks_x,
        ticktext=ticks_text,
        ticklabelstandoff=30,
        automargin=True,
        tickfont=dict(size=14)
    ),
    template="plotly_white",
    legend=dict(orientation='h', y=-0.1, x=0.5, xanchor='center'),
    margin=dict(l=80, r=40, t=100, b=120)
)



# Mostrar gráfico
fig.show()
fig.write_html("rentabilidad_acumulada_2021_2025.html")



In [ ]:
import pandas as pd
import plotly.graph_objects as go

# -------------------------------------------------
# 1. Leer archivo Excel
# -------------------------------------------------
file_path = "/content/drive/MyDrive/FondosdePensiones/rentabilidad_real_mensual_fondos_deflactada_uf.xls"
df = pd.read_excel(file_path, skiprows=3, engine="xlrd")

# -------------------------------------------------
# 2. Renombrar columnas
# -------------------------------------------------
df.columns = ['Fecha_raw','A','B','C','D','E']

# -------------------------------------------------
# 3. Convertir fechas y eliminar filas inválidas
# -------------------------------------------------
df['Fecha_dt'] = pd.to_datetime(df['Fecha_raw'], errors='coerce')
df = df.dropna(subset=['Fecha_dt'])

# -------------------------------------------------
# 4. Pasar rentabilidades de % a decimal
# -------------------------------------------------
fondos = ['A','B','C','D','E']
df[fondos] = df[fondos] / 100

# -------------------------------------------------
# 5. Filtrar período de análisis
# -------------------------------------------------
df = df[(df['Fecha_dt'] >= '2002-09-01') & (df['Fecha_dt'] <= '2025-12-31')]

# -------------------------------------------------
# 6. Calcular índice acumulado normal
# -------------------------------------------------
df_idx = df.copy()
for f in fondos:
    df_idx[f] = (1 + df_idx[f]).cumprod()

# -------------------------------------------------
# 7. Forzar septiembre 2002 = 1 para todos los fondos
# -------------------------------------------------
base_date = pd.to_datetime("2002-09-01")
mask_base = df_idx['Fecha_dt'].dt.to_period("M") == base_date.to_period("M")

# Forzar manualmente que todos valgan 1 en ese mes
for f in fondos:
    df_idx.loc[mask_base, f] = 1

# Reescalar toda la serie usando ese punto como base
base_vals = df_idx.loc[mask_base, fondos].iloc[0]
for f in fondos:
    df_idx[f] = df_idx[f] / base_vals[f]

# -------------------------------------------------
# 8. Definir ticks: cada 2 años (enero de años impares)
# -------------------------------------------------
ticks_x = df_idx[
    (df_idx['Fecha_dt'].dt.month == 6) &
    (df_idx['Fecha_dt'].dt.year % 2 == 1)
]['Fecha_dt']
ticks_text = ticks_x.dt.year.astype(str)

# -------------------------------------------------
# 9. Crear gráfico Plotly
# -------------------------------------------------
fig = go.Figure()

nombres = {
    'A':'Fondo A',
    'B':'Fondo B',
    'C':'Fondo C',
    'D':'Fondo D',
    'E':'Fondo E'
}

# Agregar líneas usando loop (misma estructura para todos)
for f in fondos:
    fig.add_trace(go.Scatter(
        x=df_idx['Fecha_dt'],
        y=df_idx[f],
        mode='lines',
        name=nombres[f],
        hovertemplate=f"{nombres[f]}<br>Fecha: %{{x|%b-%Y}}<br>Índice: %{{y:.2f}}<extra></extra>"
    ))

# -------------------------------------------------
# 10. Formato del gráfico
# -------------------------------------------------
fig.update_layout(
    height=800,
    width=1200,
    title=dict(
        text="Rentabilidad Histórica Real Acumulada (Índice base Sep-2002 = 1)<br>Período septiembre 2002 - diciembre 2025",
        x=0.5,
        xanchor="center",
        font=dict(size=22)
    ),
    yaxis=dict(
        range=[0, 4],
        autorange=False,
        showgrid=True,
        zeroline=False,
        ticklabelstandoff=10,
        tickfont=dict(size=14),
        tickformat=".2f"      # dos decimales en eje Y
    ),
    xaxis=dict(
        tickmode="array",
        tickvals=ticks_x,
        ticktext=ticks_text,
        ticklabelstandoff=30,
        automargin=True,
        tickfont=dict(size=14)
    ),
    template="plotly_white",
    legend=dict(
        orientation='h',
        y=-0.1,
        x=0.5,
        xanchor='center'
    ),
    margin=dict(l=80, r=40, t=100, b=120)
)

fig.show()
# Guardar gráfico como HTML para GitHub Pages
fig.write_html("indice_fondos_base_sep2002.html")


In [ ]:
import numpy as np

resumen = []

for f in fondos:
    r = df[f]

    # Rentabilidad anual
    rent_total = (1+r).prod() - 1
    n_anios = len(r)/12
    rent_anual = (1+rent_total)**(1/n_anios) - 1

    # Volatilidad anual
    vol_anual = r.std() * np.sqrt(12)

    # Eficiencia retorno/riesgo
    eficiencia = rent_anual / vol_anual

    # Drawdown
    curva = (1+r).cumprod()
    max_prev = curva.cummax()
    drawdown = (curva/max_prev - 1).min()

    # Meses positivos
    pct_positivos = (r > 0).mean()

    resumen.append([
        f,
        rent_anual*100,
        vol_anual*100,
        eficiencia,
        drawdown*100,
        pct_positivos*100
    ])

tabla = pd.DataFrame(resumen, columns=[
    "Fondo",
    "Rentabilidad anual %",
    "Volatilidad anual %",
    "Eficiencia",
    "Max Drawdown %",
    "% Meses positivos"
]).round(2)

tabla


,Fondo,Rentabilidad anual %,Volatilidad anual %,Eficiencia,Max Drawdown %,% Meses positivos
0,A,3.75,9.69,0.39,-25.27,58.33
1,B,3.10,7.88,0.39,-19.58,56.67
2,C,1.14,7.02,0.16,-19.18,51.67
3,D,-0.02,8.60,-0.00,-20.78,51.67
4,E,0.06,9.76,0.01,-18.76,55.00


In [ ]:
fig.write_html("acumulada.html")


In [ ]:
fig2.write_html("indice.html")


In [ ]:
import pandas as pd

pd.set_option("display.float_format", "{:,.2f}".format)

# ----------------------------------------
# 1. Archivo y configuración
# ----------------------------------------
file_path = "/content/drive/MyDrive/FondosdePensiones/activos_fondos_pensiones.xls"

fondos_map = {
    "A":"Activos Fondo Tipo A",
    "B":"Activos Fondo Tipo B",
    "C":"Activos Fondo Tipo C",
    "D":"Activos Fondo Tipo D",
    "E":"Activos Fondo Tipo E",
}

afps = ["CAPITAL","CUPRUM","HABITAT","MODELO","PLANVITAL","PROVIDA","UNO"]

fecha_2025 = pd.to_datetime("2025-12-01")
fecha_2024 = pd.to_datetime("2024-12-01")

# ----------------------------------------
# 2. Utilidades
# ----------------------------------------
def make_unique(cols):
    seen = {}
    out = []
    for c in cols:
        if c not in seen:
            seen[c] = 1
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}_{seen[c]}")
    return out

def leer_fondo(sheet):
    df = pd.read_excel(file_path, sheet_name=sheet, engine="xlrd", header=None)
    fila_header = df[df[0].astype(str).str.contains("FECHA", na=False)].index[0]
    headers = df.iloc[fila_header].astype(str).str.strip().str.upper().tolist()
    headers = make_unique(headers)
    data = df.iloc[fila_header+1:].copy()
    data.columns = headers
    data = data.rename(columns={headers[0]:"FECHA"})
    data["FECHA"] = pd.to_datetime(data["FECHA"], errors="coerce")
    for c in data.columns:
        if c != "FECHA":
            data[c] = data[c].replace("-", pd.NA)
            data[c] = pd.to_numeric(data[c], errors="coerce")
    return data

# ----------------------------------------
# 3. Leer hojas por fondo
# ----------------------------------------
fondos = {k: leer_fondo(v) for k,v in fondos_map.items()}

# ----------------------------------------
# 4. Filtrar diciembre 2025 y 2024
# ----------------------------------------
f25 = {k: df[df["FECHA"]==fecha_2025] for k,df in fondos.items()}
f24 = {k: df[df["FECHA"]==fecha_2024] for k,df in fondos.items()}

# ----------------------------------------
# 5. Armar tabla por AFP
# ----------------------------------------
tabla = pd.DataFrame({"AFP": afps})

for f in fondos_map.keys():
    colvals = []
    for afp in afps:
        cols = [c for c in f25[f].columns if c.startswith(afp)]
        if cols:
            colvals.append(f25[f][cols].sum(axis=1).values[0])
        else:
            colvals.append(pd.NA)
    tabla[f] = colvals

# ----------------------------------------
# 6. Total por AFP
# ----------------------------------------
tabla["TOTAL"] = tabla[list(fondos_map.keys())].sum(axis=1, skipna=True)

# ----------------------------------------
# 7. Variación 12 meses por AFP
# ----------------------------------------
var12 = []
for i,afp in enumerate(afps):
    tot25 = tabla.loc[i,"TOTAL"]
    tot24 = 0
    for f in fondos_map.keys():
        cols = [c for c in f24[f].columns if c.startswith(afp)]
        if cols:
            tot24 += f24[f][cols].sum(axis=1).values[0]
    if pd.notna(tot25) and tot24>0:
        var12.append(100*(tot25/tot24-1))
    else:
        var12.append(pd.NA)

tabla["Variación 12m (%)"] = var12

# ----------------------------------------
# 8. Fila TOTAL (suma por columnas)
# ----------------------------------------
fila_total = {
    "AFP": "TOTAL",
    "A": tabla["A"].sum(),
    "B": tabla["B"].sum(),
    "C": tabla["C"].sum(),
    "D": tabla["D"].sum(),
    "E": tabla["E"].sum(),
    "TOTAL": tabla["TOTAL"].sum(),
    "Variación 12m (%)": None
}
tabla = pd.concat([tabla, pd.DataFrame([fila_total])], ignore_index=True)

# ----------------------------------------
# 9. Fila Variación total 12 meses (%) correcta
# ----------------------------------------
fila_var = {"AFP": "Variación total 12 meses (%)"}

for f in ["A","B","C","D","E"]:
    v25 = f25[f]["SISTEMA"].values[0]
    v24 = f24[f]["SISTEMA"].values[0]
    fila_var[f] = 100*(v25/v24 - 1)

tot25 = sum(f25[f]["SISTEMA"].values[0] for f in ["A","B","C","D","E"])
tot24 = sum(f24[f]["SISTEMA"].values[0] for f in ["A","B","C","D","E"])
fila_var["TOTAL"] = 100*(tot25/tot24 - 1)
fila_var["Variación 12m (%)"] = None

tabla = pd.concat([tabla, pd.DataFrame([fila_var])], ignore_index=True)

# ----------------------------------------
# 10. Rellenar NaN de fila TOTAL con variación del sistema
# ----------------------------------------


var_sistema = tabla.loc[tabla["AFP"]=="Variación total 12 meses (%)","TOTAL"].values[0]
tabla.loc[tabla["AFP"]=="TOTAL","Variación 12m (%)"] = var_sistema

# ----------------------------------------
# 11. Resultado final
# ----------------------------------------
tabla


/tmp/ipython-input-2677886028.py:115: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipython-input-2677886028.py:132: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,AFP,A,B,C,D,E,TOTAL,Variación 12m (%)
0,CAPITAL,"7,508,841.80","8,096,165.28","12,698,024.16","7,567,465.22","4,979,128.99","40,849,625.45",16.41
1,CUPRUM,"8,394,899.27","7,799,436.07","12,570,425.28","4,727,723.90","4,330,701.08","37,823,185.61",14.53
2,HABITAT,"10,875,954.09","10,281,795.26","22,145,302.89","9,409,136.70","5,925,497.14","58,637,686.07",15.62
3,MODELO,"3,021,833.04","7,548,373.52","4,670,981.53","1,037,460.66","1,999,405.85","18,278,054.60",32.25
4,PLANVITAL,"1,942,952.08","4,913,282.83","4,081,166.74","1,665,654.74","1,476,140.33","14,079,196.71",30.93
5,PROVIDA,"5,674,729.77","8,206,638.07","14,666,430.61","11,648,300.14","4,145,793.37","44,341,891.96",13.54
6,UNO,"760,240.49","1,811,111.33","880,035.98","201,467.25","392,213.78","4,045,068.83",63.38
7,TOTAL,"38,179,450.54","48,656,802.36","71,712,367.18","36,257,208.61","23,248,880.54","218,054,709.23",17.91
8,Variación total 12 meses (%),23.09,26.34,14.93,15.52,7.52,17.91,NaN


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from urllib.parse import urljoin

# =================================================
# CONFIG
# =================================================
BASE_URL = "https://www.spensiones.cl"
PERIODO = "202512"
OUTPUT_DIR = "data"

os.makedirs(OUTPUT_DIR, exist_ok=True)

session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.spensiones.cl/apps/centroEstadisticas/paginaCuadrosCCEE.php"
})

# =================================================
# 1️⃣ Página intermedia
# =================================================
intermediate_url = (
    f"{BASE_URL}/apps/loadCarteras/loadCarAgr.php"
    f"?menu=sci&menuN1=estfinfp&menuN2=NOID"
    f"&orden=20&periodo={PERIODO}&ext=.php"
)

resp = session.get(intermediate_url)
resp.raise_for_status()

# =================================================
# 2️⃣ Link real
# =================================================
soup = BeautifulSoup(resp.text, "html.parser")

download_link = None
for a in soup.find_all("a", href=True):
    if "genera_xsl2xls.php" in a["href"]:
        download_link = urljoin(BASE_URL, a["href"])
        break

if not download_link:
    raise RuntimeError("No se encontró link de descarga")

# =================================================
# 3️⃣ Descargar HTML (el falso XLS)
# =================================================
html_resp = session.get(download_link)
html_resp.raise_for_status()

html_path = os.path.join(OUTPUT_DIR, f"cartera_agregada_{PERIODO}.html")
with open(html_path, "wb") as f:
    f.write(html_resp.content)

print("HTML guardado:", html_path)

# =================================================
# 4️⃣ PARSEAR HTML A DATAFRAME
# =================================================
tables = pd.read_html(html_path, flavor="bs4")
df_raw = tables[0]

# Limpieza básica
df_raw = df_raw.dropna(how="all")
df_raw.columns = [str(c).strip() for c in df_raw.columns]

new_cols = []

fondos = ["A", "B", "C", "D", "E"]
metricas = ["MMUSD", "PCT"]

for i, col in enumerate(df_raw.columns):

    col_str = str(col)

    # Primera columna: tipo instrumento
    if i == 0 or "Unnamed" in col_str:
        new_cols.append("TipoInstrumento")
        continue

    # i-1 porque saltamos TipoInstrumento
    idx = i - 1

    fondo = fondos[(idx // 2) % len(fondos)]
    metrica = metricas[idx % 2]

    new_cols.append(f"Fondo_{fondo}_{metrica}")

df_raw.columns = new_cols

#

HTML guardado: data/cartera_agregada_202512.html


In [12]:
df_raw.head()

,"('Unnamed: 0_level_0', 'Unnamed: 0_level_1')","('A', 'MMUS$')","('A', '%Fondo')","('B', 'MMUS$')","('B', '%Fondo')","('C', 'MMUS$')","('C', '%Fondo')","('D', 'MMUS$')","('D', '%Fondo')","('E', 'MMUS$')","('E', '%Fondo')","('TOTAL', 'MMUS$')","('TOTAL', '%Fondo')"
0,INVERSIÓN NACIONAL TOTAL,"7.850,46",1874,"17.997,52",3370,"38.615,92",4907,"28.764,95",7229,"22.520,92",8826,"115.750,25",4837
1,RENTA VARIABLE,"6.355,36",1517,"6.691,12",1253,"8.434,00",1072,"2.517,82",633,53398,209,"24.532,28",1025
2,Acciones,"6.123,90",1462,"6.457,24",1209,"8.024,94",1020,"2.378,33",598,53371,209,"23.518,12",983
3,Fondos de Inversión y Otros (4),19067,046,20250,038,31179,040,12325,031,000,000,82820,035
4,Activos Alternativos (5),4079,010,3138,006,9727,012,1624,004,027,000,18595,008


In [22]:
df_raw

,TipoInstrumento,Fondo_A_MMUSD,Fondo_A_PCT,Fondo_B_MMUSD,Fondo_B_PCT,Fondo_C_MMUSD,Fondo_C_PCT,Fondo_D_MMUSD,Fondo_D_PCT,Fondo_E_MMUSD,Fondo_E_PCT,Fondo_A_MMUSD,Fondo_A_PCT
0,INVERSIÓN NACIONAL TOTAL,"7.850,46",1874,"17.997,52",3370,"38.615,92",4907,"28.764,95",7229,"22.520,92",8826,"115.750,25",4837
1,RENTA VARIABLE,"6.355,36",1517,"6.691,12",1253,"8.434,00",1072,"2.517,82",633,53398,209,"24.532,28",1025
2,Acciones,"6.123,90",1462,"6.457,24",1209,"8.024,94",1020,"2.378,33",598,53371,209,"23.518,12",983
3,Fondos de Inversión y Otros (4),19067,046,20250,038,31179,040,12325,031,000,000,82820,035
4,Activos Alternativos (5),4079,010,3138,006,9727,012,1624,004,027,000,18595,008
5,Instrumentos Letra K (11),-,-,-,-,-,-,-,-,-,-,-,-
6,RENTA FIJA,"1.184,87",283,"10.860,47",2034,"29.328,98",3727,"25.756,43",6473,"21.799,24",8544,"88.929,99",3716
7,Instrumentos Banco Central,10496,025,15092,028,8952,011,10053,025,5605,022,50198,021
8,Instrumentos Tesorería,26988,064,"4.541,72",851,"16.863,84",2143,"14.799,87",3719,"11.166,15",4376,"47.641,46",1991
9,Bonos de Reconocimiento y MINVU,090,000,7731,014,5130,007,2903,007,4426,017,20280,008


In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from urllib.parse import urljoin

# =================================================
# CONFIG
# =================================================
BASE_URL = "https://www.spensiones.cl"
PERIODO = "202512"
OUTPUT_DIR = "data"

os.makedirs(OUTPUT_DIR, exist_ok=True)

session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.spensiones.cl/apps/centroEstadisticas/paginaCuadrosCCEE.php"
})

# =================================================
# 1️⃣ Página intermedia
# =================================================
intermediate_url = (
    f"{BASE_URL}/apps/loadCarteras/loadCarAgr.php"
    f"?menu=sci&menuN1=estfinfp&menuN2=NOID"
    f"&orden=20&periodo={PERIODO}&ext=.php"
)

resp = session.get(intermediate_url, timeout=30)
resp.raise_for_status()

# =================================================
# 2️⃣ Buscar link real
# =================================================
soup = BeautifulSoup(resp.text, "html.parser")

download_link = None
for a in soup.find_all("a", href=True):
    if "genera_xsl2xls.php" in a["href"]:
        download_link = urljoin(BASE_URL, a["href"])
        break

if not download_link:
    raise RuntimeError("No se encontró link de descarga")

# =================================================
# 3️⃣ Descargar HTML (XLS falso)
# =================================================
html_resp = session.get(download_link, timeout=60)
html_resp.raise_for_status()

html_path = os.path.join(OUTPUT_DIR, f"cartera_agregada_{PERIODO}.html")
with open(html_path, "wb") as f:
    f.write(html_resp.content)

print("✅ HTML guardado:", html_path)

# =================================================
# 4️⃣ Parsear HTML a DataFrame
# =================================================
tables = pd.read_html(html_path, flavor="bs4")
if not tables:
    raise RuntimeError("No se encontraron tablas en el HTML")

df_raw = tables[0]

# Limpieza mínima
df_raw = df_raw.dropna(how="all").reset_index(drop=True)

# =================================================
# 5️⃣ Renombrar columnas (A–E por posición)
# =================================================
new_cols = []

fondos = ["A", "B", "C", "D", "E"]
metricas = ["MMUSD", "PCT"]

for i, col in enumerate(df_raw.columns):

    # Primera columna
    if i == 0:
        new_cols.append("TipoInstrumento")
        continue

    idx = i - 1
    fondo = fondos[(idx // 2) % len(fondos)]
    metrica = metricas[idx % 2]

    new_cols.append(f"Fondo_{fondo}_{metrica}")

df_raw.columns = new_cols

# =================================================
# 6️⃣ NORMALIZAR SEPARADORES NUMÉRICOS (ROBUSTO)
# =================================================
import numpy as np

def limpiar_numero(x):
    if pd.isna(x):
        return np.nan
    x = str(x)
    x = x.replace(".", "")   # quitar miles
    x = x.replace(",", ".")  # coma decimal → punto
    try:
        return float(x)
    except ValueError:
        return np.nan

cols_numericas = [c for c in df_raw.columns if c != "TipoInstrumento"]

for col in cols_numericas:
    bloque = df_raw[col]

    # Caso 1: columna normal (Series)
    if bloque.ndim == 1:
        df_raw[col] = bloque.apply(limpiar_numero)

    # Caso 2: columnas duplicadas (DataFrame)
    else:
        df_raw[col] = bloque.applymap(limpiar_numero)



# =================================================
# 7️⃣ Guardar CSV limpio
# =================================================
csv_path = os.path.join(OUTPUT_DIR, f"cartera_agregada_{PERIODO}.csv")

df_raw.to_csv(
    csv_path,
    index=False,
    decimal="."
)

print("✅ CSV limpio guardado en:", csv_path)
print("Tipos de datos finales:")
print(df_raw.dtypes)


✅ HTML guardado: data/cartera_agregada_202512.html
✅ CSV limpio guardado en: data/cartera_agregada_202512.csv
Tipos de datos finales:
TipoInstrumento     object
Fondo_A_MMUSD      float64
Fondo_A_PCT        float64
Fondo_B_MMUSD      float64
Fondo_B_PCT        float64
Fondo_C_MMUSD      float64
Fondo_C_PCT        float64
Fondo_D_MMUSD      float64
Fondo_D_PCT        float64
Fondo_E_MMUSD      float64
Fondo_E_PCT        float64
Fondo_A_MMUSD      float64
Fondo_A_PCT        float64
dtype: object


/tmp/ipython-input-1530915824.py:123: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_raw[col] = bloque.applymap(limpiar_numero)


In [29]:
df_raw

,TipoInstrumento,Fondo_A_MMUSD,Fondo_A_PCT,Fondo_B_MMUSD,Fondo_B_PCT,Fondo_C_MMUSD,Fondo_C_PCT,Fondo_D_MMUSD,Fondo_D_PCT,Fondo_E_MMUSD,Fondo_E_PCT,Fondo_A_MMUSD,Fondo_A_PCT
0,INVERSIÓN NACIONAL TOTAL,785046.0,18740.0,17997.52,3370.0,38615.92,4907.0,28764.95,7229.0,22520.92,8826.0,11575025.0,48370.0
1,RENTA VARIABLE,635536.0,15170.0,6691.12,1253.0,8434.00,1072.0,2517.82,633.0,53398.00,209.0,2453228.0,10250.0
2,Acciones,61239.0,14620.0,6457.24,1209.0,8024.94,1020.0,2378.33,598.0,53371.00,209.0,2351812.0,9830.0
3,Fondos de Inversión y Otros (4),190670.0,460.0,20250.00,38.0,31179.00,40.0,12325.00,31.0,0.00,0.0,828200.0,350.0
4,Activos Alternativos (5),40790.0,100.0,3138.00,6.0,9727.00,12.0,1624.00,4.0,27.00,0.0,185950.0,80.0
5,Instrumentos Letra K (11),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,RENTA FIJA,118487.0,2830.0,10860.47,2034.0,29328.98,3727.0,25756.43,6473.0,21799.24,8544.0,8892999.0,37160.0
7,Instrumentos Banco Central,104960.0,250.0,15092.00,28.0,8952.00,11.0,10053.00,25.0,5605.00,22.0,501980.0,210.0
8,Instrumentos Tesorería,269880.0,640.0,4541.72,851.0,16863.84,2143.0,14799.87,3719.0,11166.15,4376.0,4764146.0,19910.0
9,Bonos de Reconocimiento y MINVU,900.0,0.0,7731.00,14.0,5130.00,7.0,2903.00,7.0,4426.00,17.0,202800.0,80.0


In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from urllib.parse import urljoin

# =================================================
# CONFIG
# =================================================
BASE_URL = "https://www.spensiones.cl"
PERIODO = "202512"
OUTPUT_DIR = "data"

os.makedirs(OUTPUT_DIR, exist_ok=True)

session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.spensiones.cl/apps/centroEstadisticas/paginaCuadrosCCEE.php"
})

# =================================================
# 1️⃣ Página intermedia
# =================================================
intermediate_url = (
    f"{BASE_URL}/apps/loadCarteras/loadCarAgr.php"
    f"?menu=sci&menuN1=estfinfp&menuN2=NOID"
    f"&orden=20&periodo={PERIODO}&ext=.php"
)

resp = session.get(intermediate_url, timeout=30)
resp.raise_for_status()

# =================================================
# 2️⃣ Buscar link real
# =================================================
soup = BeautifulSoup(resp.text, "html.parser")

download_link = None
for a in soup.find_all("a", href=True):
    if "genera_xsl2xls.php" in a["href"]:
        download_link = urljoin(BASE_URL, a["href"])
        break

if not download_link:
    raise RuntimeError("No se encontró link de descarga")

# =================================================
# 3️⃣ Descargar HTML (XLS falso)
# =================================================
html_resp = session.get(download_link, timeout=60)
html_resp.raise_for_status()

html_path = os.path.join(OUTPUT_DIR, f"cartera_agregada_{PERIODO}.html")
with open(html_path, "wb") as f:
    f.write(html_resp.content)

print("✅ HTML guardado:", html_path)

# =================================================
# 4️⃣ Parsear HTML → TODO COMO STRING
# =================================================
df_raw = pd.read_html(
    html_path,
    flavor="bs4"
)[0]



✅ HTML guardado: data/cartera_agregada_202512.html


In [41]:
# Limpieza mínima
df_raw = df_raw.dropna(how="all").reset_index(drop=True)

# =================================================
# 5️⃣ Renombrar columnas (A–E por posición)
# =================================================
new_cols = []

fondos = ["A", "B", "C", "D", "E"]
metricas = ["MMUSD", "PCT"]

for i, col in enumerate(df_raw.columns):

    # Primera columna
    if i == 0:
        new_cols.append("TipoInstrumento")
        continue

    idx = i - 1
    fondo = fondos[(idx // 2) % len(fondos)]
    metrica = metricas[idx % 2]

    new_cols.append(f"Fondo_{fondo}_{metrica}")

df_raw.columns = new_cols

# =================================================
# 6️⃣ NORMALIZAR SEPARADORES NUMÉRICOS (ROBUSTO)
# =================================================
import numpy as np

def limpiar_numero(x):
    if pd.isna(x):
        return np.nan
    x = str(x)
    x = x.replace(".", "")   # quitar miles
    x = x.replace(",", ".")  # coma decimal → punto
    try:
        return float(x)
    except ValueError:
        return np.nan

cols_numericas = [c for c in df_raw.columns if c != "TipoInstrumento"]

for col in cols_numericas:
    # Ensure the column is treated as a Series before applying
    df_raw[col] = df_raw[col].apply(limpiar_numero)


print("Tipos de datos finales después de la limpieza:")
print(df_raw.dtypes)
display(df_raw.head())

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [38]:
df_raw

Unnamed: 0_level_0          A                 B  \
                         Unnamed: 0_level_1      MMUS$ %Fondo      MMUS$   
0                  INVERSIÓN NACIONAL TOTAL   7.850,46   1874  17.997,52   
1                            RENTA VARIABLE   6.355,36   1517   6.691,12   
2                                  Acciones   6.123,90   1462   6.457,24   
3           Fondos de Inversión y Otros (4)      19067    046      20250   
4                  Activos Alternativos (5)       4079    010       3138   
5                 Instrumentos Letra K (11)          -      -          -   
6                                RENTA FIJA   1.184,87    283  10.860,47   
7                Instrumentos Banco Central      10496    025      15092   
8                    Instrumentos Tesorería      26988    064   4.541,72   
9           Bonos de Reconocimiento y MINVU        090    000       7731   
10  Bonos de Empresas y Efectos de Comercio      16435    039   2.017,68   
11   Bonos emitidos por Fondos de Inversión        011    000        264   
12                          Bonos Bancarios      28228    067   3.591,88   
13                      Letras Hipotecarias        238    001        609   
14                        Depósitos a Plazo          -      -       2260   
15         Fondos Mutuos y de Inversión (6)       8672    021      11250   
16                 Activos Alternativos (7)      26322    063      31979   
17                Instrumentos Letra K (10)          -      -          -   
18                            Disponible(1)       1006    002       1734   
19                                DERIVADOS      28718    069      32850   
20                      OTROS NACIONALES(2)       2306    006      11742   
21               INVERSIÓN EXTRANJERA TOTAL  34.050,64   8126  35.402,25   
22                           RENTA VARIABLE  27.463,15   6554  25.432,84   
23                            Fondos Mutuos  12.565,56   2999  11.256,86   
24                 Activos Alternativos (8)   2.662,12    635   2.520,33   
25                                    Otros  12.235,47   2920  11.655,64   
26                           RENTA FIJA (9)   6.241,86   1490   9.507,51   
27                                DERIVADOS      25627    061      36276   
28                     OTROS EXTRANJEROS(3)       8937    021       9914   
29                            TOTAL ACTIVOS  41.901,11  10000  53.399,77   
30                  SUBTOTAL RENTA VARIABLE  33.818,52   8071  32.123,97   
31                      SUBTOTAL RENTA FIJA   7.426,73   1772  20.367,98   
32                       SUBTOTAL DERIVADOS      54344    130      69126   
33                           SUBTOTAL OTROS      11242    027      21656   

                   C                 D                 E              TOTAL  \
   %Fondo      MMUS$ %Fondo      MMUS$ %Fondo      MMUS$ %Fondo       MMUS$   
0    3370  38.615,92   4907  28.764,95   7229  22.520,92   8826  115.750,25   
1    1253   8.434,00   1072   2.517,82    633      53398    209   24.532,28   
2    1209   8.024,94   1020   2.378,33    598      53371    209   23.518,12   
3     038      31179    040      12325    031        000    000       82820   
4     006       9727    012       1624    004        027    000       18595   
5       -          -      -          -      -          -      -           -   
6    2034  29.328,98   3727  25.756,43   6473  21.799,24   8544   88.929,99   
7     028       8952    011      10053    025       5605    022       50198   
8     851  16.863,84   2143  14.799,87   3719  11.166,15   4376   47.641,46   
9     014       5130    007       2903    007       4426    017       20280   
10    378   4.758,75    605   4.824,88   1213   4.911,05   1925   16.676,71   
11    000       1682    002       3969    010       3985    016        9912   
12    673   6.571,11    835   5.489,58   1380   5.193,40   2035   21.128,24   
13    001       2320    003       1169    003        806    003        5141   
14    004       2194    